In [76]:
#importing necessary libraries
import pandas as pd
from  sklearn.neighbors import NearestNeighbors
import numpy as np
import pickle
from ingredient_parser_function import ingredient_parser
import unidecode
from sklearn.metrics.pairwise import cosine_similarity

In [77]:
def ingredient_parser_final(ingredient):
    if isinstance(ingredient, list):
        ingredients = ingredient
    else:
        # Split the string into individual ingredients
        ingredients = ingredient.split(',')
    
    ingredients = ','.join(ingredients)
    ingredients = unidecode.unidecode(ingredients)
    return ingredients

def title_parser(title):
    title = unidecode.unidecode(title)
    return title 



In [78]:
def recs(ingredients):
    df_recipes = pd.read_csv('C:/Users/ACER/Desktop/Recipe_Recommendation/jupyter notebooks/parsed_data.csv')

    # Loading ML model and encoding (encoding is basically the mapping of letters to numbers)
    with open('C:/Users/ACER/Desktop/Recipe_Recommendation/jupyter notebooks/encoding.pickle', 'rb') as f:
        tfidf_encodings = pickle.load(f)
        
    with open('C:/Users/ACER/Desktop/Recipe_Recommendation/jupyter notebooks/trained_model.pickle', "rb") as f:
        tfidf = pickle.load(f)
        
    try:
        ingredients_parsed = ingredient_parser(ingredients)
    except (ValueError, TypeError) as e:
        ingredients_parsed = ingredient_parser([ingredients])
        
    if isinstance(ingredients_parsed, str):
        ingredients_parsed = [ingredients_parsed]
        
    ingredients_tfidf = tfidf.transform(ingredients_parsed) #transforms the input according to my tfidf model
    
    knn = NearestNeighbors(n_neighbors=5, metric='cosine') #Initializes knn algorithm to find 5 nearest neighbours based on cosine similarity
    knn.fit(tfidf_encodings)
    distances, indices = knn.kneighbors(ingredients_tfidf) #applies the knn algorithm, the output has a 2d array containing the distances of the points from the input and a list containing the indices of the nearest points

    # Use a list to store recommendations
    recommendations = []

    for i in indices[0]:
        title = title_parser(df_recipes['Title'][i])
        ingredients = ingredient_parser_final(df_recipes['Parsed_Ingredients'][i])
        instructions = title_parser(df_recipes['Instructions'][i])
        recommendations.append({'Title': title, 'Ingredients': ingredients, 'Instructions': instructions})

    # Create the DataFrame from the list
    recommendation = pd.DataFrame(recommendations)
    
    #adding distances to the dataframe
    scores = [distances[0][i] for i in range(len(distances[0]))]    
    recommendation['Scores']= scores

    return recommendation


In [79]:
test_ingredients = "dough, onions, tomato " 
rec = recs(test_ingredients)

print(rec)

                                      Title               Ingredients  \
0  Flatbread with Onions and Mustard Seeds   bread dough onion garlic   
1              Phyllo Onion and Dill Purses        phyllo dough onion   
2        Anne Willan's Home Canned Tomatoes              tomato onion   
3                       Fennel-Tomato Salad              tomato onion   
4                          The Meza's Salsa              onion tomato   

                                        Instructions    Scores  
0  Place dough in large bowl. Cover with towel an...  0.284312  
1  Preheat oven to 350 degrees.\nIn a large saute...  0.426352  
2  Pack whole tomatoes into quart/litre jars with...  0.427111  
3  Slice ingredients into medium-sized slices. Se...  0.427111  
4  Peel leaves off of the cilantro and set aside....  0.427111  
